Run on: Main database
Create Tables on Main Database, if they do not already exist.  
If they do exist already, empty them.

In [ ]:
USE Salesianer;
GO

IF OBJECT_ID(N'__SDCTouren') IS NULL
BEGIN
  CREATE TABLE __SDCTouren (
    TourenID int
  );
END ELSE BEGIN
  TRUNCATE TABLE __SDCTouren;
END;

IF OBJECT_ID(N'__SDCVsa') IS NULL
BEGIN
  CREATE TABLE __SDCVsa (
    VsaID int
  );
END ELSE BEGIN
  TRUNCATE TABLE __SDCVsa;
END;

IF OBJECT_ID(N'__SDCTraeger') IS NULL
BEGIN
  CREATE TABLE __SDCTraeger (
    TraegerID int
  );
END ELSE BEGIN
  TRUNCATE TABLE __SDCTraeger;
END;

IF OBJECT_ID(N'__SDCTraeArti') IS NULL
BEGIN
  CREATE TABLE __SDCTraeArti (
    TraeArtiID int
  );
END ELSE BEGIN
  TRUNCATE TABLE __SDCTraeArti;
END;

IF OBJECT_ID(N'__SDCTeile') IS NULL
BEGIN
  CREATE TABLE __SDCTeile (
    TeileID int
  );
END ELSE BEGIN
  TRUNCATE TABLE __SDCTeile;
END;

IF OBJECT_ID(N'__SDCHinweis') IS NULL
BEGIN
  CREATE TABLE __SDCHinweis (
    HinweisID int
  );
END ELSE BEGIN
  TRUNCATE TABLE __SDCHinweis;
END;

IF OBJECT_ID(N'__SDCJahrLief') IS NULL
BEGIN
  CREATE TABLE __SDCJahrLief (
    TourenID int
  );
END ELSE BEGIN
  TRUNCATE TABLE __SDCJahrLief;
END;

GO

Run on: SDC database!  
**Replace database in USE-statement and set variable to correct SdcDevID!**
This writes all missing rows into the tables created on the main database in the first step.

In [3]:
USE Salesianer_SAWR;
GO

IF OBJECT_ID('tempdb..#SdcDiff') IS NOT NULL
  DROP TABLE #SdcDiff;

GO

DECLARE @SdcDevID int = 51;

CREATE TABLE #SdcDiff (
  TableName nvarchar(20),
  Differences int
);

INSERT INTO [SALADVPSQLC1A1.salres.com].Salesianer.dbo.__SdcTouren
SELECT ZentralTouren.ID
FROM [SALADVPSQLC1A1.salres.com].Salesianer.dbo.Touren AS ZentralTouren
WHERE NOT EXISTS (
  SELECT Touren.ID
  FROM Touren
  WHERE Touren.ID = ZentralTouren.ID
);

INSERT INTO #SdcDiff VALUES (N'Touren', @@ROWCOUNT);

INSERT INTO [SALADVPSQLC1A1.salres.com].Salesianer.dbo.__SdcVsa
SELECT ZentralVSA.ID
FROM [SALADVPSQLC1A1.salres.com].Salesianer.dbo.Vsa AS ZentralVSA
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.StandBer ON StandBer.StandKonID = ZentralVSA.StandKonID
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.StBerSDC ON StBerSDC.StandBerID = StandBer.ID
WHERE StBerSDC.SdcDevID = @SdcDevID
  AND NOT EXISTS (
    SELECT Vsa.ID
    FROM Vsa
    WHERE Vsa.ID = ZentralVSA.ID
  );

INSERT INTO #SdcDiff VALUES (N'Vsa', @@ROWCOUNT);

INSERT INTO [SALADVPSQLC1A1.salres.com].Salesianer.dbo.__SdcVsaTour
SELECT ZentralVSATour.ID
FROM [SALADVPSQLC1A1.salres.com].Salesianer.dbo.VsaTour AS ZentralVSATour
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.Vsa ON ZentralVSATour.VsaID = Vsa.ID
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.StandBer ON StandBer.StandKonID = Vsa.StandKonID
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.StBerSDC ON StBerSDC.StandBerID = StandBer.ID
WHERE StBerSDC.SdcDevID = @SdcDevID
  AND NOT EXISTS (
    SELECT VsaTour.*
    FROM VsaTour
    WHERE VsaTour.ID = ZentralVSATour.ID
  );

INSERT INTO #SdcDiff VALUES (N'VsaTour', @@ROWCOUNT);

INSERT INTO [SALADVPSQLC1A1.salres.com].Salesianer.dbo.__SdcTraeger
SELECT ZentralTraeger.ID
FROM [SALADVPSQLC1A1.salres.com].Salesianer.dbo.Traeger AS ZentralTraeger
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.Vsa ON ZentralTraeger.VsaID = Vsa.ID
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.StandBer ON StandBer.StandKonID = Vsa.StandKonID
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.StBerSDC ON StBerSDC.StandBerID = StandBer.ID
WHERE StBerSDC.SdcDevID = @SdcDevID
  AND NOT EXISTS (
    SELECT Traeger.*
    FROM Traeger
    WHERE Traeger.ID = ZentralTraeger.ID
  );

INSERT INTO #SdcDiff VALUES (N'Traeger', @@ROWCOUNT);

INSERT INTO [SALADVPSQLC1A1.salres.com].Salesianer.dbo.__SdcTraeArti
SELECT ZentralTraeArti.ID
FROM [SALADVPSQLC1A1.salres.com].Salesianer.dbo.TraeArti AS ZentralTraeArti
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.Traeger ON ZentralTraeArti.TraegerID = Traeger.ID
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.Vsa ON Traeger.VsaID = Vsa.ID
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.StandBer ON StandBer.StandKonID = Vsa.StandKonID
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.StBerSDC ON StBerSDC.StandBerID = StandBer.ID
WHERE StBerSDC.SdcDevID = @SdcDevID
  AND NOT EXISTS (
    SELECT TraeArti.*
    FROM TraeArti
    WHERE TraeArti.ID = ZentralTraeArti.ID
  );

INSERT INTO #SdcDiff VALUES (N'TraeArti', @@ROWCOUNT);

INSERT INTO [SALADVPSQLC1A1.salres.com].Salesianer.dbo.__SdcTeile
SELECT ZentralTeile.ID
FROM [SALADVPSQLC1A1.salres.com].Salesianer.dbo.Teile AS ZentralTeile
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.Vsa ON ZentralTeile.VsaID = Vsa.ID
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.StandBer ON StandBer.StandKonID = Vsa.StandKonID
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.StBerSDC ON StBerSDC.StandBerID = StandBer.ID
WHERE StBerSDC.SdcDevID = @SdcDevID
  AND ZentralTeile.AltenheimModus = 0
  AND NOT EXISTS (
    SELECT Teile.*
    FROM Teile
    WHERE Teile.ID = ZentralTeile.ID
  );

INSERT INTO #SdcDiff VALUES (N'Teile', @@ROWCOUNT);

INSERT INTO [SALADVPSQLC1A1.salres.com].Salesianer.dbo.__SdcHinweis
SELECT ZentralHinweis.ID
FROM [SALADVPSQLC1A1.salres.com].Salesianer.dbo.Hinweis AS ZentralHinweis
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.Teile ON ZentralHinweis.TeileID = Teile.ID
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.Vsa ON Teile.VsaID = Vsa.ID
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.StandBer ON StandBer.StandKonID = Vsa.StandKonID
JOIN [SALADVPSQLC1A1.salres.com].Salesianer.dbo.StBerSDC ON StBerSDC.StandBerID = StandBer.ID
WHERE StBerSDC.SdcDevID = @SdcDevID
  AND Teile.AltenheimModus = 0
  AND NOT EXISTS (
    SELECT Hinweis.*
    FROM Hinweis
    WHERE Hinweis.ID = ZentralHinweis.ID
      AND Hinweis.Aktiv = ZentralHinweis.Aktiv
  );

INSERT INTO #SdcDiff VALUES (N'Hinweis', @@ROWCOUNT);

/*INSERT INTO [SALADVPSQLC1A1.salres.com].Salesianer.dbo.__SdcJahrLief
SELECT ZentralJahrLief.ID
FROM [SALADVPSQLC1A1.salres.com].Salesianer.dbo.JahrLief AS ZentralJahrLief
WHERE NOT EXISTS (
  SELECT JahrLief.ID
  FROM JahrLief
  WHERE JahrLief.ID = ZentralJahrLief.ID
);

INSERT INTO #SdcDiff VALUES (N'JahrLief', @@ROWCOUNT);*/

SELECT * FROM #SdcDiff;

GO

Commands completed successfully.

Commands completed successfully.

Run on: Main database

This pseudo-updates the missing rows from step 2. The triggers on the tables are run and the rows are sent to the replication queue to be sent to the SDC database.

In [ ]:
USE Salesianer;
GO

UPDATE Touren SET Tour = Tour
WHERE Touren.ID IN (
  SELECT TourenID FROM __SDCTouren
);

GO

UPDATE Vsa SET SuchCode = SuchCode
WHERE Vsa.ID IN (
  SELECT VsaID FROM __SDCVsa
);

GO

UPDATE VsaTour SET Folge = Folge
WHERE VsaTour.ID IN (
  SELECT VsaTourID FROM __SDCVsaTour
);

GO

UPDATE Traeger SET Traeger = Traeger
WHERE Traeger.ID IN (
  SELECT TraegerID FROM __SDCTraeger
);

GO

UPDATE TraeArti SET Menge = Menge
WHERE TraeArti.ID IN (
  SELECT TraeArtiID FROM __SDCTraeArti
);

GO

UPDATE Teile SET RuecklaufG = RuecklaufG
WHERE Teile.ID IN (
  SELECT TeileID FROM __SDCTeile
);

GO

UPDATE Hinweis SET Aktiv = Aktiv
WHERE Hinweis.ID IN (
  SELECT HinweisID FROM __SDCHinweis
);

GO

UPDATE JahrLief SET Lieferwochen = Lieferwochen
WHERE JahrLief.ID IN (
  SELECT JahrLiefID FROM __SDCJahrLief
);

GO

Run on: Main database
Replication process can be monitored with the following script

In [ ]:
USE Salesianer;
GO

SELECT SdcDev.ID AS SdcDevID, SdcDev.Bez AS Sortieranlage, RepQueue.TableName, RepQueue.Typ, COUNT(DISTINCT RepQueue.TableID) AS AnzahlDS
FROM RepQueue
JOIN SdcDev ON RepQueue.SdcDevID = SdcDev.ID
WHERE RepQueue.ApplicationID = N'AdvanTex.exe'
GROUP BY SdcDev.ID, SdcDev.Bez, RepQueue.TableName, RepQueue.Typ
ORDER BY SdcDevID;

GO